This notebook will study the convergence and "error bars" for the accuracy of $f_N^*$, the optimal function for noisy inputs, by evaluating these functions on finite datasets.

The current experiment for counterexamples trains an SAN to find the lowest error function on a training (or validation!) set, call this $\hat{f}_{train}$ or $\hat{f}_{test}$. The definition of $\hat{f}_{test}$ is that it minimizes error on a _specific_ test set, call it $D_N$ where $N$ is the number of data. This immediately gives two statistical fluctuations:
 1. For different $D_N$, we get different $\hat{f}_{test}$
 2. Any specific $\hat{f}_{test}$ will achieve different accuracies when evaluated on different $D_N$

We want to show the (error, sensitivity) point that $\hat{f}_{test}$ is _attracted towards_. What does this mean? Suppose that we have a function $h$ that we are computing (error, sens) for.
 - this coordinate should be(?) specific for the dataset: $\hat{f}$ cannot "see" alternative datasets.
 - This coordinate could therefore be the error of the $h$ that achieves optimal performance on $D_N$
 - This coordinate _might not_ be the same as the coordinate for $f_N^*$
 - A plausible choice for $h$ should be the optimal function for the specific dataset Matheus is looking at
 - **problem**: Every time i resample a dataset, I might get a pretty different sensitivity. **however** I expect all of these sensitivities to be smaller than sens of $f$, right?

In [6]:
# autoreload magic
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from mindreadingautobots.sequence_generators import make_datasets, data_io
import numpy as np
from mindreadingautobots.entropy_and_bayesian import boolean
import itertools

counterexample = "00100110"

In [152]:

p_bitflip = .2
n_train = 10000
signature = (0, 0, 0, 1, 1, 0, 0, 0, 0)
n_bits = len(signature) - 1  # n_bits is total number of bits in X, not including the label
subseq_idx = list(range(n_bits)) # for this exercise, we don't want/need subsets
signature = dict(zip(range(len(signature)), signature))
k = n_bits

all_bitstrings = np.array(list(itertools.product([0, 1], repeat=n_bits)))
true_func = lambda x: signature[sum(x)]
true_sens = boolean.average_sensitivity(true_func, all_bitstrings)
print(f"true sensitivity: {true_sens}")
# seed = 1234

err_memorize_Ztrain_arr = []
sens_memorize_Ztrain_arr = []
for seed in list(1234 + np.arange(10)):
    # sample a variety of "training performances"
    # this loop will find the f* point for the training data. To do it for the test data, we just re-run with the num data equal to val size
    X, Z, subseq_idx = make_datasets.sparse_boolean_weightbased_k_n(n_bits + 1, k, n_train, signature, p_bitflip=p_bitflip, seed=seed, subseq_idx=subseq_idx)
    f_lookup_on_Z, lookup_table_on_Z = boolean.dataset_lookup_table(Z)
    f_lookup_on_X, lookup_table_on_X = boolean.dataset_lookup_table(X)
    err_memorize_Ztrain_evaluate_Ztrain = 1 - boolean.compute_acc_on_dataset(f_lookup_on_Z, Z)
    sens_memorize_Ztrain = boolean.average_sensitivity(f_lookup_on_Z, all_bitstrings)

    err_memorize_Ztrain_arr.append(err_memorize_Ztrain_evaluate_Ztrain)
    sens_memorize_Ztrain_arr.append(sens_memorize_Ztrain)
    # training_data_memorization_X_err = 1 - boolean.compute_acc_on_dataset(f_lookup_on_Z, X)


true sensitivity: 3.5


In [153]:
print(err_memorize_Ztrain_arr)
print(sens_memorize_Ztrain_arr)


[0.3642000000000537, 0.36210000000005393, 0.36640000000005346, 0.3596000000000542, 0.3649000000000536, 0.37180000000005287, 0.3632000000000538, 0.3681000000000533, 0.3650000000000536, 0.36550000000005356]
[3.015625, 2.890625, 3.0625, 2.875, 2.859375, 2.96875, 3.0625, 3.0625, 3.0, 2.984375]


In [106]:
boolean.compute_acc_test(f_lookup, true_func, n_bits)

0.92578125